In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("users.csv")

/var/folders/h1/t4ps1by94ml46mqx3tjkych80000gn/T/ipykernel_30121/218062245.py:4: DtypeWarning: Columns (1,4,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("users.csv")


In [2]:
df

,_id,blocked,country,role,adminRole,freeTrial,local,ip,position,profileFilled,...,countryCode,emailLogs,smsLogs,delete,tutorial,creditCard,privacyPolicyAccepted,directMarketingAccepted,createdAt,updatedAt
0,577df5a89ff623fb74e01ea5,NaN,Georgia,admin,590ca3a9e86e3d4fe1f34f65,"{'isUsed': True, 'usefulUntil': datetime.datet...",{'email': 'v.veliashvili@gmail.com'},176.221.205.131,1,True,...,ge,[{'emailId': b'Y\xa5\\\x91\xb5\xf4\x87es\xfb\x...,"[{'smsId': b'ZC\xbfm\x11Bh\x19\xff\x1c\xb6;', ...","{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': False, 'hint': False, 'audi...","{'isAttached': True, 'attachDate': datetime.da...",True,True,2016-07-07 06:24:40.226,2023-10-17 12:02:32.804
1,577df6399ff623fb74e01eab,NaN,Georgia,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': 'testirakla@lingwing.com'},176.221.205.131,2,True,...,ge,NaN,NaN,"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...",NaN,NaN,NaN,2016-07-07 06:27:05.907,2020-02-27 20:40:45.007
2,577df6489ff623fb74e01eac,NaN,Georgia,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': 'goga@lingwing.com'},176.221.205.131,3,True,...,ge,NaN,NaN,"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...",NaN,NaN,NaN,2016-07-07 06:27:20.969,2020-02-27 20:40:45.009
3,577e3ac5fe68aa60b080486f,NaN,Georgia,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': 'samanishvilis@dedinacvali.hoo'},92.241.86.122,4,True,...,ge,NaN,NaN,"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...",NaN,NaN,NaN,2016-07-07 11:19:33.887,2020-02-27 20:40:45.012
4,577e3b02fe68aa60b0804870,NaN,Georgia,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': 'otara@gmail.com'},92.241.86.122,5,True,...,ge,NaN,NaN,"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...",NaN,NaN,NaN,2016-07-07 11:20:34.124,2020-02-27 20:40:45.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285989,652f41f8819efa0f2bf10195,NaN,Georgia,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': 'lanagachechiladze2002@gmail.com'},5.152.113.75,285990,False,...,ge,[{'emailId': b'Y\xc9t\xe2sh\x14^\x87c\xfb\x85'...,[],"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...","{'isAttached': False, 'attachDate': None}",True,False,2023-10-18 02:28:25.044,2023-10-18 02:28:25.687
285990,652f6f5d819efa0f2bf106a0,NaN,NaN,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': None},94.43.51.207,285991,False,...,NaN,[{'emailId': b'Y\xc9t\xe2sh\x14^\x87c\xfb\x85'...,[],"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...","{'isAttached': False, 'attachDate': None}",True,False,2023-10-18 05:38:37.517,2023-10-18 05:48:49.885
285991,652f7bab819efa0f2bf10bb0,NaN,NaN,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': None},31.146.115.34,285992,False,...,NaN,[],[],"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': False, 'hint': False, 'audi...","{'isAttached': False, 'attachDate': None}",True,False,2023-10-18 06:34:45.912,2023-10-18 06:34:45.919
285992,652f8c86819efa0f2bf114fd,NaN,NaN,regular,NaN,"{'isUsed': False, 'usefulUntil': None}",{'email': 'rtveladzemari@gmail.com'},31.146.70.140,285993,True,...,NaN,[],[],"{'isDeleted': 'false', 'deleteDate': None, 'us...","{'test': {'intro': True, 'hint': True, 'audio'...","{'isAttached': False, 'attachDate': None}",True,False,2023-10-18 07:43:02.777,2023-10-18 07:43:58.440


In [5]:
import pandas as pd
import json

def is_jsonable(x):
    try:
        json.dumps(x)
        return True
    except (TypeError, OverflowError):
        return False

def recursive_normalize(data, prefix=''):
    # Handle strings that are JSON objects
    if isinstance(data, str) and is_jsonable(data):
        data = json.loads(data)

    # If data is a dictionary, apply json_normalize and then recursively normalize each column
    if isinstance(data, dict):
        result = json_normalize(data)
        for col in result.columns:
            result[col] = result[col].apply(lambda x: recursive_normalize(x, prefix=f'{prefix}{col}.'))
        return result

    # If data is a list, apply the function to each item in the list
    if isinstance(data, list):
        return pd.concat([recursive_normalize(item, prefix) for item in data], ignore_index=True)

    # For other types, just return the data
    return data

def normalize_dataframe(df, nested_cols):
    for col in nested_cols:
        if col in df.columns:
            # Apply recursive normalization
            expanded_data = df[col].apply(lambda x: recursive_normalize(x, prefix=f'{col}.'))
            # Concatenate the normalized data
            expanded_data = pd.concat(expanded_data.tolist(), ignore_index=True)
            df = df.drop(columns=[col]).join(expanded_data, how='left')
    return df


In [7]:
nested_cols = ['freeTrial', 'local', 'facebook', 'google', 'twitter', 'info', 'profile', 'emailLogs', 'smsLogs', 'delete', 'tutorial', 'creditCard']
df_normalized = normalize_dataframe(df, nested_cols)


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)